In [1]:
import numpy as np
import text_classifier as tclf

from sklearn.model_selection import KFold

import sys
sys.path.append("..")
import utils

In [5]:
spam_train, spam_test = utils.load_data('spam')
clf = tclf.TextClassifier(spam_train['texts'], spam_train['labels'], embed_dim=2)

1013lines [00:00, 10123.90lines/s]Constructing vocabulary...
4457lines [00:00, 10156.83lines/s]
100%|██████████| 4457/4457 [00:00<00:00, 4719.19it/s]
Training model...
Epoch: 1  | time in 0 minutes, 1 seconds
	Loss: 0.0246(train)	|	Acc: 85.8%(train)
Epoch: 2  | time in 0 minutes, 1 seconds
	Loss: 0.0098(train)	|	Acc: 94.1%(train)
Epoch: 3  | time in 0 minutes, 2 seconds
	Loss: 0.0040(train)	|	Acc: 98.3%(train)
Epoch: 4  | time in 0 minutes, 2 seconds
	Loss: 0.0022(train)	|	Acc: 99.3%(train)
Epoch: 5  | time in 0 minutes, 2 seconds
	Loss: 0.0014(train)	|	Acc: 99.6%(train)



In [16]:
clf.train_dataset

           48316,  52576,   1863,   9474,  64270,  93921,  78688,  99684,  29152,
           77319,  79476,  27686,  35848,  17843,  89471,  37520,  61089,  94988,
           76315,  57734,  93863,  71090,  22379,  87007])),
 (0,
  tensor([     4,     35,      8,     44,    640,      7,  39824,    122,     28,
             474,      2,      4,     35,   1580,      6,     39,      6,  15194,
               8,     44,   1580,     19,   2676,      2,     37,      4,    694,
               6,    128,      2,    164,    280,     55,  86422,  84638,  26125,
           39825,  18959,  17065,   1240,     24,    164,  43239,  65624,  23782,
            1705,  23793,  15195,     55,   7025,  65617,  19183,   5281,    245,
             266,  58906,  43039,   1836,    735,    536,    281,  25535,  86423,
           84639,  26126,  39826,  18960,  17066,  10442,   1173,  58924,  43240,
           65625, 100900,  42246,  23794,  15196,   5467,  86348,  65618,  19184,
           22995,   1867,  83804

In [19]:
def cross_validate(X, y):
    k = 3
    kf = KFold(k)
    dims = [1, 10]
    results = {D: [] for D in dims}

    for D in dims:
        for train_idx, val_idx in kf.split(X):
            X_train, X_val = X[train_idx], X[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]

            clf = text_clf.TextClassifier(X_train, y_train, embed_dim=D)

            predictions = [clf.predict(text) for text in X_val]
            acc = (predictions == y_val).sum() / y_val.shape[0]
            results[D].append(acc)
    results = {D: np.mean(accs) for D, accs in results.items()}
    return results

In [20]:
accuracires = cross_validate(spam_train['texts'], spam_train['labels'])

682lines [00:00, 6819.50lines/s]Constructing vocabulary...
2971lines [00:00, 7916.65lines/s]
100%|██████████| 2971/2971 [00:00<00:00, 5337.76it/s]
Training model...
Epoch: 1  | time in 0 minutes, 1 seconds
	Loss: 0.0257(train)	|	Acc: 85.7%(train)
Epoch: 2  | time in 0 minutes, 1 seconds
	Loss: 0.0176(train)	|	Acc: 88.2%(train)
Epoch: 3  | time in 0 minutes, 1 seconds
	Loss: 0.0080(train)	|	Acc: 95.8%(train)
Epoch: 4  | time in 0 minutes, 1 seconds
	Loss: 0.0042(train)	|	Acc: 98.3%(train)
Epoch: 5  | time in 0 minutes, 1 seconds
	Loss: 0.0027(train)	|	Acc: 99.1%(train)

746lines [00:00, 7452.81lines/s]Constructing vocabulary...
2971lines [00:00, 7332.63lines/s]
100%|██████████| 2971/2971 [00:00<00:00, 5249.43it/s]
Training model...
Epoch: 1  | time in 0 minutes, 1 seconds
	Loss: 0.0256(train)	|	Acc: 85.8%(train)
Epoch: 2  | time in 0 minutes, 1 seconds
	Loss: 0.0143(train)	|	Acc: 90.4%(train)
Epoch: 3  | time in 0 minutes, 1 seconds
	Loss: 0.0062(train)	|	Acc: 97.3%(train)
Epoch: 4  | t

{1: 0.9535554437752735, 10: 0.937857564730602}